In [3]:
import pygame as pg
from sys import exit
import random
from pygame import mixer
import time

class Player:
    def __init__(self, screen, img, fps, x, y):
        self.screen = screen
        self.count = 0
        self.fps = fps
        self.img = img
        self.index = 0
        self.x = x
        self.y = y
        self.image = self.img[self.index]
        self.update()
 
    def update(self):
        self.count += 1
        if self.index >= len(self.img):
            self.index = 0
        self.image = self.img[self.index]
        if self.count > self.fps:
            self.index += 1
            self.count = 0
            
    def is_clicked(self, pos):
        self.rect = self.image.get_rect(center = (self.x,self.y))
        return self.rect.collidepoint(pos)

    def draw(self):
        self.update()
        self.screen.blit(self.image, self.image.get_rect(center = (self.x,self.y)))

class Button:
    def __init__(self, x, y, width, height, text, bg_color, text_color, action, font_size=32):
        self.rect = pg.Rect(x, y, width, height)
        self.text = text
        self.bg_color = bg_color
        self.text_color = text_color
        self.action = action
        self.hovered = False
        self.font = pg.font.Font('Pixeltype.ttf', font_size)
        self.txt_surface = self.font.render(text, True, self.text_color)
        self.shadow_color = pg.Color('gray10')
        self.shadow_offset = (3, 3)

    def draw(self, screen):
        if self.hovered:
            self.color = pg.Color('dodgerblue2')
        else:
            self.color = self.bg_color
        
        # Create a surface for the button with alpha transparency
        button_surface = pg.Surface((self.rect.width, self.rect.height), flags=pg.SRCALPHA)
        # Draw the button with rounded corners
        pg.draw.rect(button_surface, self.color, self.rect, border_radius=10)
        # Draw the button shadow
        shadow_rect = self.rect.move(self.shadow_offset)
        pg.draw.rect(button_surface, self.shadow_color, shadow_rect, border_radius=10)
        # Blit the button surface onto the screen
        screen.blit(button_surface, self.rect)
        
        # Draw the button text
        text_rect = self.txt_surface.get_rect(center=self.rect.center)
        screen.blit(self.txt_surface, text_rect)

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)
    

class Game:
    def __init__(self):
        # init interface
        pg.init()
        self.screen = pg.display.set_mode((500,700)) 
        pg.display.set_caption('plankton') 
        self.clock = pg.time.Clock() 
        self.running = True

        # init status
        self.status = 'Plankton' # 'Crownfish', 'Kraken'
        self.gauge = 0 
        self.death_count = 0
        self.dead = False

        # init events
        self.feeding = False
        self.adventure = False
        self.view_history = False

        self.i = 0
        self.height = 1      

    def new(self):
         # Set player and background objects
        self.plankton = Player(self.screen, [pg.image.load('graphics/plankton1.png').convert_alpha(),
                pg.image.load('graphics/plankton2.png').convert_alpha()], 30, 250, 350)
        self.crownfish = Player(self.screen, [pg.image.load('graphics/crownfish1.png').convert_alpha(),
                pg.image.load('graphics/crownfish2.png').convert_alpha()], 30, 250, 350)
        self.kraken = Player(self.screen, [pg.image.load('graphics/fish1.png').convert_alpha(),
                pg.image.load('graphics/fish2.png').convert_alpha()], 30, 250, 350)

        self.fish1 = Player(self.screen, [pg.image.load('graphics/fish1.png').convert_alpha(),
                pg.image.load('graphics/fish2.png').convert_alpha(), 
                pg.image.load('graphics/fish3.png').convert_alpha(),
                pg.image.load('graphics/fish2.png').convert_alpha()], 20, 150, 220)
        self.fish2 = Player(self.screen, [pg.image.load('graphics/fish1.png').convert_alpha(),
                pg.image.load('graphics/fish2.png').convert_alpha(), 
                pg.image.load('graphics/fish3.png').convert_alpha(),
                pg.image.load('graphics/fish2.png').convert_alpha()], 20, 500, 500)
        self.seahorse = Player(self.screen, [pg.image.load('graphics/seahorse1.png').convert_alpha(),
                pg.image.load('graphics/seahorse2.png').convert_alpha()], 20, 100, 400)
        self.angfish1 = Player(self.screen, [pg.image.load('graphics/angfishleft1.png').convert_alpha(),
                pg.image.load('graphics/angfishleft2.png').convert_alpha()], 20, 300, 160)
        self.angfish2 = Player(self.screen, [pg.image.load('graphics/angfishright1.png').convert_alpha(),
                pg.image.load('graphics/angfishright2.png').convert_alpha()], 20, 130, 560)
        
        # Set button properties
        self.button_width = 160
        self.button_height = 139
        self.button_margin = (self.screen.get_width() - (3*self.button_width))/4
        self.button_pos_x = self.screen.get_width() - (3*self.button_width + 4*self.button_margin)

        self.button1_bg = Player(self.screen, [pg.image.load('graphics/button.png').convert_alpha()], 700, self.button_pos_x + self.button_margin + 0.5*self.button_width, 630)
        self.button2_bg = Player(self.screen, [pg.image.load('graphics/button.png').convert_alpha()], 700, self.button_pos_x + 1.5*self.button_width + 2*self.button_margin, 630)
        self.button3_bg = Player(self.screen, [pg.image.load('graphics/button.png').convert_alpha()], 700, self.button_pos_x + 2.5*self.button_width + 3*self.button_margin, 630)
        self.bar_bg = Player(self.screen, [pg.image.load('graphics/bar.png').convert_alpha()], 700, 250, 150)

        # Create buttons
        self.button1 = Button(self.button_pos_x + self.button_margin, 565, self.button_width, self.button_height, "Feed", pg.Color('#dfdfdf'), pg.Color('white'), self.start_game, 40)
        self.button2 = Button(self.button_pos_x + self.button_width + 2*self.button_margin, 565, self.button_width, self.button_height, "Adventure", pg.Color('#dfdfdf'), pg.Color('white'), self.start_game, 30)
        self.button3 = Button(self.button_pos_x + 2*self.button_width + 3*self.button_margin + 2, 565, self.button_width, self.button_height, "History", pg.Color('#dfdfdf'), pg.Color('white'), self.start_game, 38)
       
        self.run()

    def run(self):
        # BGM
        mixer.music.load('bgm/default.MID')
        mixer.music.play(-1)
        
        self.playing = True
        while self.playing:
            self.clock.tick(60)
            self.events()
            self.update()
            self.draw()

    def events(self):
        for event in pg.event.get():
            # check for closing window
            if event.type == pg.QUIT:
                if self.playing:
                    self.playing = False
                self.running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                pos = pg.mouse.get_pos()
                if self.button1.is_clicked(pos):
                    bgm = mixer.Sound("bgm/click.wav")
                    bgm.play()
                    self.feeding = True
                if self.button2.is_clicked(pos):
                    bgm = mixer.Sound("bgm/click.wav")
                    bgm.play()
                    self.adventure = True
                if self.button3.is_clicked(pos):
                    bgm = mixer.Sound("bgm/click.wav")
                    bgm.play()
                    self.view_history = True

    def update(self):
        if self.gauge >= 0 and self.gauge < 200:
            self.status = 'Plankton'
        elif self.gauge >= 200 and self.gauge < 500:
            self.status = 'CrownFish'
        elif self.gauge >= 500:
            self.status = 'Kraken'

        if self.dead:
            pg.mixer.music.pause()
            self.die()
            pg.mixer.music.unpause()
            self.dead = False

    def draw(self):
        # background animation
        background = pg.image.load('graphics/bg.png').convert()
        self.screen.blit(background, (self.i,0))
        self.screen.blit(background, (500+self.i,0))
        if self.i == -500:
            self.i=0
        self.i-=0.3

        self.fish1.draw()
        self.fish2.draw()
        self.seahorse.draw()

        if self.status == 'Plankton':
            self.plankton.draw()
        elif self.status == 'CrownFish':
            self.crownfish.draw()
        elif self.status == 'Kraken':
            self.kraken.draw()

        self.fish1.x -= 1
        if self.fish1.x == -30:
            self.fish1.x = 800

        self.fish2.x -= 1.5
        if self.fish2.x <= -30:
            self.fish2.x = 800

        self.seahorse.x += 0.5
        if self.seahorse.x >= 730:
            self.seahorse.x = 0

        self.button1_bg.draw()
        self.button2_bg.draw()
        self.button3_bg.draw()
        self.bar_bg.draw()

        self.button1.draw(self.screen)
        self.button2.draw(self.screen)
        self.button3.draw(self.screen)

        # draw title text
        font = pg.font.Font('Pixeltype.ttf', 120)
        score_surface = font.render('Plankton', False, (28,125,174)) # text, anti-alising, color
        score_rect = score_surface.get_rect(center = (250,80))

        # draw status bar
        font2 = pg.font.Font('Pixeltype.ttf', 31)
        status_surface = font2.render('Status: ' + self.status +
                                    '       Evloution gauge: ' + str(self.gauge) + "%", False, 'black') # text, anti-alising, color
        status_rect = status_surface.get_rect(center = (250,150))

        self.screen.blit(score_surface, score_rect)
        self.screen.blit(status_surface, status_rect)

        if self.feeding:
            self.feed()

        if self.adventure:
            self.move_sea()

        if self.view_history:
            self.history()

        pg.display.flip()
        
    def feed(self):
        # background
        feed_surface = pg.image.load('graphics/feed.png').convert_alpha()
        feed_rect = feed_surface.get_rect(center = (250,350))
        self.screen.blit(feed_surface, feed_rect)

        # draw text
        font = pg.font.Font('Pixeltype.ttf',55)
        text_surface = font.render('Select the feed', False, (28,125,225)) # text, anti-alising, color
        text_rect = text_surface.get_rect(center = (250,200))
        self.screen.blit(text_surface, text_rect)

        # draw feed
        food1 = Player(self.screen, [pg.image.load('graphics/fish2.png').convert_alpha()], 20, 145, 300)
        food2 = Player(self.screen, [pg.image.load('graphics/food2.png').convert_alpha()], 20, 250, 300)
        food3 = Player(self.screen, [pg.image.load('graphics/food3.png').convert_alpha()], 20, 350, 300)
        food1.draw()
        food2.draw()
        food3.draw()

        # draw exit button
        clickX = Player(self.screen, [pg.image.load('graphics/clickXbutt.png').convert_alpha()], 700, 420, 130)
        clickX.draw()

        # draw random button 
        random_butt = Button(190, 400, 120, 80, "Random feed", pg.Color('dodgerblue2'), pg.Color('white'), self.start_game, 32)
        random_butt.draw(self.screen)
        
        for event in pg.event.get():
            if event.type == pg.QUIT:
                if self.playing:
                    self.playing = False
                self.running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                pos = pg.mouse.get_pos()
                if food1.is_clicked(pos):
                    bgm = mixer.Sound("bgm/eat.wav")
                    bgm.play()
                    self.gauge += 5 
                    self.feeding = False
                if food2.is_clicked(pos):
                    bgm = mixer.Sound("bgm/eat.wav")
                    bgm.play()
                    self.gauge += 20 
                    self.feeding = False
                if food3.is_clicked(pos):
                    bgm = mixer.Sound("bgm/eat.wav")
                    bgm.play()
                    self.gauge += 50 
                    self.feeding = False
                if clickX.is_clicked(pos):
                    bgm = mixer.Sound("bgm/clickX.wav")
                    bgm.play()
                    self.feeding = False
                if random_butt.is_clicked(pos):
                    bgm = mixer.Sound("bgm/eat.wav")
                    bgm.play()
        
        pg.display.update()        

    def move_sea(self):
        # background
        feed_surface = pg.image.load('graphics/feed.png').convert_alpha()
        feed_rect = feed_surface.get_rect(center = (250,350))
        self.screen.blit(feed_surface, feed_rect)

        # draw text
        font = pg.font.Font('Pixeltype.ttf',55)
        text_surface = font.render('Select Adventure', False, (28,125,225)) # text, anti-alising, color
        text_rect = text_surface.get_rect(center = (250,200))
        self.screen.blit(text_surface, text_rect)

        deepsea_butt = Button(145, 250, 200, 80, "Swim lower", pg.Color('#dfdfdf'), pg.Color('white'), self.start_game, 32)
        shallowsea_butt = Button(145, 350, 200, 80, "Swim higher", pg.Color('#dfdfdf'), pg.Color('white'), self.start_game, 32)

        deepsea_butt.draw(self.screen)
        shallowsea_butt.draw(self.screen)

        # draw exit button
        clickX = Player(self.screen, [pg.image.load('graphics/clickXbutt.png').convert_alpha()], 700, 420, 130)
        clickX.draw()

        for event in pg.event.get():
            if event.type == pg.QUIT:
                if self.playing:
                    self.playing = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                pos = pg.mouse.get_pos()
                if deepsea_butt.is_clicked(pos):
                    bgm = mixer.Sound("bgm/click.wav")
                    bgm.play()
                    self.animate()
                    self.deepsea()
                    pg.mixer.music.unpause()
                    self.adventure = False
                elif shallowsea_butt.is_clicked(pos):
                    bgm = mixer.Sound("bgm/click.wav")
                    bgm.play()
                    self.animate()
                    self.shallowsea()
                    pg.mixer.music.unpause()
                    self.adventure = False
                if clickX.is_clicked(pos):
                    bgm = mixer.Sound("bgm/clickX.wav")
                    bgm.play()
                    self.adventure = False

        pg.display.update()    

    def history(self):
        # background
        history_surface = pg.image.load('graphics/feed.png').convert_alpha()
        history_rect = history_surface.get_rect(center = (250,350))
        self.screen.blit(history_surface, history_rect)

        # draw text
        font = pg.font.Font('Pixeltype.ttf',55)
        death_count = str(self.death_count)
        text_surface = font.render('DEATH  Count', False, (28,125,225)) 
        text_rect = text_surface.get_rect(center = (250,200))
        self.screen.blit(text_surface, text_rect)

        count_surface = font.render(death_count, False, (28,125,225)) 
        count_rect = text_surface.get_rect(center = (350,360))
        self.screen.blit(count_surface, count_rect)

        # draw exit button
        clickX = Player(self.screen, [pg.image.load('graphics/clickXbutt.png').convert_alpha()], 700, 420, 130)
        clickX.draw()

        for event in pg.event.get():
            if event.type == pg.QUIT:
                if self.playing:
                    self.playing = False
                self.running = False
            elif event.type == pg.MOUSEBUTTONDOWN:
                pos = pg.mouse.get_pos()
                if clickX.is_clicked(pos):
                    bgm = mixer.Sound("bgm/clickX.wav")
                    bgm.play()
                    self.view_history = False
                    
        pg.display.update()  

    def shallowsea(self):
        bgm = mixer.Sound("bgm/shallowsea.wav")
        bgm.play()

        count = 0
        start_time = time.time()
            
        while count <200:
            # background animation
            background = pg.image.load('graphics/shallowsea2.png').convert()
            self.screen.blit(background, (0,0))
            sea = pg.image.load('graphics/shallowsea1.png').convert_alpha()
            self.screen.blit(sea, (self.i,0))
            self.screen.blit(sea, (500+self.i,0))
            if self.i == -500:
                self.i=0
            self.i-=0.3

            self.fish2.draw()
            self.seahorse.draw()

            if self.status == 'Plankton':
                self.plankton.draw()
            elif self.status == 'CrownFish':
                self.crownfish.draw()
            elif self.status == 'Kraken':
                self.kraken.draw()

            self.fish2.x -= 1.5
            if self.fish2.x <= -30:
                self.fish2.x = 800

            self.seahorse.x += 0.5
            if self.seahorse.x >= 730:
                self.seahorse.x = 0

            # draw title text
            font = pg.font.Font('Pixeltype.ttf', 120)
            score_surface = font.render('Plankton', False, (28,125,174)) # text, anti-alising, color
            score_rect = score_surface.get_rect(center = (250,80))
            self.screen.blit(score_surface, score_rect) 
            
            # draw status bar
            self.bar_bg.draw()
            font2 = pg.font.Font('Pixeltype.ttf', 31)
            status_surface = font2.render('Status: ' + self.status +
                                    '       Evloution gauge: ' + str(self.gauge) + "%", False, 'black') # text, anti-alising, color
            status_rect = status_surface.get_rect(center = (250,150))
            self.screen.blit(status_surface, status_rect)

            self.button1_bg.draw()
            self.button2_bg.draw()
            self.button3_bg.draw()
        
            self.button1.draw(self.screen)
            self.button2.draw(self.screen)
            self.button3.draw(self.screen)              

            if time.time() - start_time > 2:
                messages = ["Dragged to saltern\nand became salt", "Became bird food", "Caught by a surfer", "Swept to death\nby tsunami waves", "Returned safely"]
                random_message = random.choice(messages)
                if(random_message == "Returned safely"):
                    self.gauge += 50
                    self.draw()
                else:
                    self.dead = True
                    self.status == 'Plankton'
                    self.gauge = 0
                    self.death_count += 1

                shallowSea_surface = pg.image.load('graphics/feed.png').convert_alpha()
                shallowSea_rect = shallowSea_surface.get_rect(center = (250,350))
                self.screen.blit(shallowSea_surface, shallowSea_rect)

                font3 = pg.font.Font('Pixeltype.ttf', 40)
                text_surface = font3.render(random_message, False, (28,125,174))

                # Calculate the position of each line of text
                lines = random_message.split('\n')
                text_rects = []
                total_height = len(lines) * font3.get_linesize()
                y = (self.screen.get_height() - total_height) // 2
                if "\n" in random_message:
                    for line in lines:
                        text_rect = text_surface.get_rect(center=(350, y)) #newline 350, 없으면 250
                        text_rects.append(text_rect)
                        y += font3.get_linesize()
                else:
                    for line in lines:
                        text_rect = text_surface.get_rect(center=(250, y)) #newline 350, 없으면 250
                        text_rects.append(text_rect)
                        y += font3.get_linesize()   

                # Draw each line of text
                for i, line in enumerate(lines): 
                    self.screen.blit(font3.render(line, False, (28, 125, 174)), text_rects[i])

                pg.display.flip()
                pg.time.delay(4000)
                break

            pg.display.flip()
            count += 1

        bgm.fadeout(3)

    def deepsea(self):
        bgm = mixer.Sound("bgm/deepsea.wav")
        bgm.play()
        
        count = 0
        start_time = time.time()

        while count <200:
            # background animation
            background = pg.image.load('graphics/deepsea.png').convert()
            self.screen.blit(background, (self.i,0))
            self.screen.blit(background, (500+self.i,0))
            if self.i == -500:
                self.i=0
            self.i-=0.3

            self.angfish1.draw()
            self.angfish2.draw()
            self.seahorse.draw()
            
            if self.status == 'Plankton':
                self.plankton.draw()
            elif self.status == 'CrownFish':
                self.crownfish.draw()
            elif self.status == 'Kraken':
                self.kraken.draw()

            self.angfish2.x += 1.5
            if self.seahorse.x >= 730:
                self.seahorse.x = 0

            self.angfish1.x -= 1.0
            if self.fish2.x <= -30:
                self.fish2.x = 800

            self.seahorse.x += 0.5
            if self.seahorse.x >= 730:
                self.seahorse.x = 0

            # draw title text
            font = pg.font.Font('Pixeltype.ttf', 120)
            score_surface = font.render('Plankton', False, (28,125,174)) # text, anti-alising, color
            score_rect = score_surface.get_rect(center = (250,80))
            self.screen.blit(score_surface, score_rect)

            # draw status bar
            self.bar_bg.draw()
            font2 = pg.font.Font('Pixeltype.ttf', 31)
            status_surface = font2.render('Status: ' + self.status +
                                    '       Evloution gauge: ' + str(self.gauge) + "%", False, 'black') # text, anti-alising, color
            status_rect = status_surface.get_rect(center = (250,150))
            self.screen.blit(status_surface, status_rect)
            
            self.button1_bg.draw()
            self.button2_bg.draw()
            self.button3_bg.draw()
            self.button1.draw(self.screen)
            self.button2.draw(self.screen)
            self.button3.draw(self.screen)

            shadow = pg.image.load('graphics/shadow.png').convert()
            shadow.set_alpha(128)
            self.screen.blit(shadow, (0,0))

            if time.time() - start_time > 2:
                messages = ["Bite the submarine\ncable and died of\nelectric shock", "Got lost in a\nsunken pirate\nship and died", "Went too deep,\ndied of pressure\ndifference", "Attacked by\nbigger creature\nand died" ,"Returned safely"]
                random_message = random.choice(messages)
                if(random_message == "Returned safely"):
                    self.gauge += 50
                    self.draw()
                else:
                    self.status == 'Plankton'
                    self.gauge = 0
                    self.death_count += 1
                deepSea_surface = pg.image.load('graphics/feed.png').convert_alpha()
                deepSea_rect = deepSea_surface.get_rect(center = (250,350))
                self.screen.blit(deepSea_surface, deepSea_rect)

                font3 = pg.font.Font('Pixeltype.ttf', 40)
                text_surface = font3.render(random_message, False, (28,125,174))

                # Calculate the position of each line of text
                lines = random_message.split('\n')
                text_rects = []
                total_height = len(lines) * font3.get_linesize()
                y = (self.screen.get_height() - total_height) // 2
                if "\n" in random_message:
                    for line in lines:
                        text_rect = text_surface.get_rect(topleft=(150, y)) #newline 350, 없으면 250
                        text_rects.append(text_rect)
                        y += font3.get_linesize()
                else:
                    for line in lines:
                        text_rect = text_surface.get_rect(center=(250, y)) #newline 350, 없으면 250
                        text_rects.append(text_rect)
                        y += font3.get_linesize()

                # Draw each line of text
                for i, line in enumerate(lines): 
                    self.screen.blit(font3.render(line, False, (28, 125, 174)), text_rects[i])

                pg.display.flip()
                pg.time.delay(4000)
                break

            pg.display.flip()
            count += 1
        bgm.fadeout(3)
    
    def animate(self):  
        pg.mixer.music.pause()
        bgm = mixer.Sound("bgm/animate.wav")
        bgm.play()

        img = [pg.image.load('graphics/animate/animate1.png').convert_alpha(),
                pg.image.load('graphics/animate/animate2.png').convert_alpha(), 
                pg.image.load('graphics/animate/animate3.png').convert_alpha(),
                pg.image.load('graphics/animate/animate4.png').convert_alpha(),
                pg.image.load('graphics/animate/animate5.png').convert_alpha(),
                pg.image.load('graphics/animate/animate6.png').convert_alpha(),
                pg.image.load('graphics/animate/animate7.png').convert_alpha(),
                pg.image.load('graphics/animate/animate8.png').convert_alpha()]

        for image in img:
            pg.time.delay(180)
            self.screen.blit(image, (0,0))
            pg.display.update()  

    def show_start_screen(self):
        # game splash/start screen      
        self.screen.fill((28,125,174))

        font = pg.font.Font('Pixeltype.ttf', 120)
        score_surface = font.render('Plankton', False, 'Black') # text, anti-alising, color
        score_rect = score_surface.get_rect(center = (250,300))

        font = pg.font.Font('Pixeltype.ttf', 40)
        score_surface1 = font.render('Press any key to start', False, 'Black') # text, anti-alising, color
        score_rect1 = score_surface1.get_rect(center = (250,340))

        self.screen.blit(score_surface, score_rect)
        self.screen.blit(score_surface1, score_rect1)
        pg.display.flip()
        self.wait_for_key()

    def wait_for_key(self):
        waiting = True
        while waiting:
            self.clock.tick(60)
            for event in pg.event.get():
                if event.type == pg.QUIT:
                    waiting = False
                    self.running = False
                if event.type == pg.KEYUP:
                    waiting = False
    
    def start_game(self):
        pass

    def die(self):
        bgm = mixer.Sound("bgm/dead.wav")
        bgm.play()
   
        self.screen.fill('Black')

        font1 = pg.font.Font('Pixeltype.ttf', 120)
        score_surface = font1.render('You Died', False, 'White') # text, anti-alising, color
        score_rect = score_surface.get_rect(center = (250,280))

        font2 = pg.font.Font('Pixeltype.ttf', 40)
        score_surface1 = font2.render('Your status was: ' + self.status, False, 'White') # text, anti-alising, color
        score_rect1 = score_surface1.get_rect(center = (250,330))

        font3 = pg.font.Font('Pixeltype.ttf', 40)
        score_surface2 = font3.render('Death Count: ' + str(self.death_count), False, 'White') # text, anti-alising, color
        score_rect2 = score_surface2.get_rect(center = (250,380))

        font4 = pg.font.Font('Pixeltype.ttf', 30)
        score_surface3 = font4.render('Press any key to restart', False, 'White') # text, anti-alising, color
        score_rect3 = score_surface3.get_rect(center = (250,430))

        self.screen.blit(score_surface, score_rect)
        self.screen.blit(score_surface1, score_rect1)
        self.screen.blit(score_surface2, score_rect2)
        self.screen.blit(score_surface3, score_rect3)
        pg.display.flip()
        self.wait_for_key()


g = Game()
g.show_start_screen()
while g.running:
    g.new()

pg.quit()

: 

: 